In [ ]:
import os
import re
import pandas as pd
import numpy as np
from collections import Counter

import spacy
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import optuna

In [ ]:
dataPath = os.path.join("data", "track-a.csv")

In [3]:
dataFrame = pd.read_csv(dataPath)

In [4]:
nlpModel = spacy.load("en_core_web_sm", disable = ["parser", "ner"])

In [5]:
textColumn = "text"
labelColumns = ["anger", "fear", "joy", "sadness", "surprise"]

In [6]:
def cleanerFunction(text: str) -> str:
    tempDoc = nlpModel(text)
    tokens = [
        tok.lemma_.lower()
        for tok in tempDoc
        if not tok.is_stop and not tok.is_punct and tok.lemma_ != "-PRON-"
    ]
    return " ".join(tokens)

In [7]:
dataFrame["Spacy_text"] = dataFrame["text"].astype(str).apply(cleanerFunction)

In [8]:
userDevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
labelColumns = [col for col in ["anger", "fear", "joy", "sadness", "surprise"] if col in dataFrame]
yData = dataFrame[labelColumns].values.astype(np.float32)

In [10]:
xAll = dataFrame["Spacy_text"].tolist()
xTrain, xVal, yTrain, yVal = train_test_split(xAll, yData, test_size = 0.1, random_state = 69)

In [11]:
xTrain = xTrain.copy()
xVal = xVal.copy()

In [12]:
numLabels = yTrain.shape[1]

In [13]:
TOKEN_PATTERN = re.compile(r"\b\w+\b")

In [14]:
def simpleTokenize(text: str) -> list:
    return TOKEN_PATTERN.findall(text.lower())

In [15]:
def vocabularyBuilder(texts: list, vocabsize: int):
    counter = Counter()
    for t in texts:
        tokens = simpleTokenize(t)
        counter.update(tokens)
    
    mostCommon = counter.most_common(vocabsize - 2)
    indextoword = ["<pad>", "<unk>"] + [token for token, _ in mostCommon]
    wordtoindex = {w: i for i, w in enumerate(indextoword)}

    return wordtoindex, indextoword

In [16]:
MAX_VOCAB_SIZE = 20000
MAX_SEQUENCE_LENGTH = 100

wordtoindex, indextoword = vocabularyBuilder(xTrain, vocabsize = MAX_VOCAB_SIZE)
vocabSize = len(indextoword)

padIndex = wordtoindex["<pad>"]
unkIndex = wordtoindex["<unk>"]

In [17]:
def encodePadFunction(texts: list, wordtoindex: dict, sequenceLength: int = MAX_SEQUENCE_LENGTH) -> np.ndarray:
    encodings = []

    for t in texts:
        tokens = simpleTokenize(t)
        tokenIDs = [wordtoindex.get(tok, unkIndex) for tok in tokens]
        if len(tokenIDs) > sequenceLength:
            tokenIDs = tokenIDs[:sequenceLength]
        else:
            tokenIDs = tokenIDs + [padIndex] * (sequenceLength - len(tokenIDs))
        encodings.append(tokenIDs)

    return np.array(encodings, dtype = np.int64)

In [18]:
trainEncode = encodePadFunction(xTrain, wordtoindex, sequenceLength = MAX_SEQUENCE_LENGTH)
valEncode = encodePadFunction(xVal, wordtoindex, sequenceLength = MAX_SEQUENCE_LENGTH)

In [19]:
class TextDataset(Dataset):
    def __init__(self, encodings: np.ndarray, labels: np.ndarray) -> None:
        self.encodings = torch.from_numpy(encodings)
        self.labels = torch.from_numpy(labels)

    def __len__(self):
        return self.encodings.size(0)
    
    def __getitem__(self, index):
        return self.encodings[index], self.labels[index]

## Feed-forward Neural Network

In [20]:
class feedforwardNeuralNetwork(nn.Module):
    def __init__(
            self,
            vocabSize: int,
            embeddingDim: int,
            hiddenUnit1: int,
            hiddenUnit2: int,
            dropoutRate: float,
            numLabels: int,
            padIndex: int
        ) -> None:
        super().__init__()
        self.embedding = nn.Embedding(vocabSize, embeddingDim, padding_idx = padIndex)
        self.dropout1 = nn.Dropout(dropoutRate)
        self.fc1 = nn.Linear(embeddingDim, hiddenUnit1)
        self.bn1 = nn.BatchNorm1d(hiddenUnit1)
        self.relu = nn.ReLU()
        self.dropout2 = nn.Dropout(dropoutRate)
        self.fc2 = nn.Linear(hiddenUnit1, hiddenUnit2)
        self.bn2 = nn.BatchNorm1d(hiddenUnit2)
        self.dropout3 = nn.Dropout(dropoutRate)
        self.fc_out = nn.Linear(hiddenUnit2, numLabels)

    def forward(self, x):
        emb = self.embedding(x)             
        avgEmb = emb.mean(dim = 1) 
        h1 = self.fc1(avgEmb)
        h1 = self.bn1(h1)
        h1 = self.relu(h1)
        h1 = self.dropout2(h1)
        h2 = self.fc2(h1)
        h2 = self.bn2(h2)
        h2 = self.relu(h2)
        h2 = self.dropout3(h2)
        return self.fc_out(h2)    

In [21]:
def epochTrain(model, dataloader, optimizer, criterion, device):
    model.train()
    totalLoss = 0.0
    for batchInputs, batchLabels in dataloader:
        batchInputs = batchInputs.to(device)
        batchLabels = batchLabels.to(device)

        optimizer.zero_grad()
        logits = model(batchInputs)
        loss = criterion(logits, batchLabels)
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        totalLoss += loss.item() * batchInputs.size(0)
    
    return totalLoss / len(dataloader.dataset)

In [22]:
def evaluateModel(model, dataloader, criterion, device):
    model.eval()
    totalLoss = 0.0
    with torch.no_grad():
        for batchInputs, batchLabels in dataloader:
            batchInputs = batchInputs.to(device)
            batchLabels = batchLabels.to(device)

            logits = model(batchInputs)
            loss = criterion(logits, batchLabels)
            totalLoss += loss.item() * batchInputs.size(0)

    return totalLoss / len(dataloader.dataset)

In [23]:
freq = np.maximum(yTrain.sum(axis = 0) / len(yTrain), 1e-4)
class_weights = 1.0 / freq
class_weights = class_weights / class_weights.sum()
weight_tensor = torch.FloatTensor(class_weights).to(userDevice)

In [24]:
def objective(trial):
    embeddingDim = trial.suggest_categorical("embeddingDim", [32, 64, 128, 256, 512,])
    hiddenUnit1 = trial.suggest_int("hiddenUnit1", 32, 256, step = 16)
    hiddenUnit2 = trial.suggest_int("hiddenUnit2", 32, 256, step = 16)
    dropoutRate = trial.suggest_float("dropoutRate", 0.2, 0.6, step = 0.1)
    learnRate = trial.suggest_float("learnRate", 1e-4, 1e-2, log = True)
    weightDecay = trial.suggest_float("weightDecay", 1e-6, 1e-2, log = True)
    batchSize = trial.suggest_categorical("batchSize", [32, 64, 128, 256])
    epochs = trial.suggest_categorical("epochs", [3, 5, 7, 9, 11, 13, 15])
    optName = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    use_scheduler = trial.suggest_categorical("use_scheduler", [True, False])
    if optName == "SGD":
        sgdvar = trial.suggest_float("momentum", 0.1, 0.9)

    trainDataset = TextDataset(trainEncode, yTrain)
    valDataset = TextDataset(valEncode, yVal)

    trainLoader = DataLoader(trainDataset, batch_size = batchSize, shuffle = True)
    valLoader = DataLoader(valDataset, batch_size = batchSize, shuffle = True)

    model = feedforwardNeuralNetwork(vocabSize, embeddingDim, hiddenUnit1, hiddenUnit2, dropoutRate, numLabels, padIndex).to(userDevice)

    if optName == "Adam":
        optimizer = optim.Adam(model.parameters(), lr = learnRate, weight_decay = weightDecay)
    elif optName == "RMSprop":
        optimizer = optim.RMSprop(model.parameters(), lr = learnRate, weight_decay = weightDecay)
    else:
        optimizer = optim.SGD(model.parameters(), lr = learnRate, momentum = sgdvar) # type: ignore

    criterionOption = nn.BCEWithLogitsLoss(pos_weight = weight_tensor)

    bestvalLoss = float("inf")
    counterVar = 0
    bestState = None

    if use_scheduler:
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = "min", factor = 0.5, patience = 1)

    for epoch in range(1, epochs + 1):
        trainLoss = epochTrain(model, trainLoader, optimizer, criterionOption, userDevice)
        valLoss = evaluateModel(model, valLoader, criterionOption, userDevice)

        if use_scheduler:
            scheduler.step(valLoss) # type: ignore

        trial.report(valLoss, epoch)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
        
        if valLoss < bestvalLoss:
            bestvalLoss = valLoss
            counterVar = 0
            bestState = {k: v.cpu() for k, v in model.state_dict().items()}
        else:
            counterVar += 1
            if counterVar >=2:
                break

    model.load_state_dict(bestState) # type: ignore
    return bestvalLoss

In [25]:
def _objective(trial):
    return objective(trial)

In [38]:
study = optuna.create_study(study_name = "FFNN", direction = "minimize", sampler = optuna.samplers.TPESampler(seed = 42), storage = "sqlite:///db.sqlite3")

[I 2025-06-10 16:58:32,056] A new study created in RDB with name: FFNN


In [39]:
study.optimize(_objective, n_trials = 500)

[I 2025-06-10 16:58:40,814] Trial 0 finished with value: 0.39435342244723215 and parameters: {'embeddingDim': 64, 'hiddenUnit1': 64, 'hiddenUnit2': 32, 'dropoutRate': 0.6, 'learnRate': 0.0015930522616241021, 'weightDecay': 0.0006796578090758161, 'batchSize': 64, 'epochs': 15, 'optimizer': 'SGD', 'use_scheduler': False, 'momentum': 0.25973902572668783}. Best is trial 0 with value: 0.39435342244723215.
[I 2025-06-10 16:58:44,504] Trial 1 finished with value: 0.22771626463435618 and parameters: {'embeddingDim': 256, 'hiddenUnit1': 32, 'hiddenUnit2': 256, 'dropoutRate': 0.6, 'learnRate': 0.004138040112561018, 'weightDecay': 1.6536937182824424e-05, 'batchSize': 64, 'epochs': 7, 'optimizer': 'SGD', 'use_scheduler': False, 'momentum': 0.815861880342119}. Best is trial 1 with value: 0.22771626463435618.
[I 2025-06-10 16:58:46,105] Trial 2 finished with value: 0.1854491994484237 and parameters: {'embeddingDim': 64, 'hiddenUnit1': 96, 'hiddenUnit2': 112, 'dropoutRate': 0.30000000000000004, 'lear

In [40]:
print("\nOptuna Best Trial:")
best = study.best_trial
print(f"Validation Loss: {best.value:.4f}")
for key, val in best.params.items():
    print(f"  {key}: {val}")
print("\n")

best_embeddingDim = best.params["embeddingDim"]
best_hiddenUnit1 = best.params["hiddenUnit1"]
best_hiddenUnit2 = best.params["hiddenUnit2"]
best_dropoutRate = best.params["dropoutRate"]
best_learnRate = best.params["learnRate"]
best_weightDecay = best.params["weightDecay"]
best_batchSize = best.params["batchSize"]
best_epochs = best.params["epochs"]
best_optimizer = best.params["optimizer"]
best_use_scheduler = best.params["use_scheduler"]
if best_optimizer == "SGD":
    best_momentum = best.params["momentum"]


Optuna Best Trial:
Validation Loss: 0.1667
  embeddingDim: 512
  hiddenUnit1: 176
  hiddenUnit2: 48
  dropoutRate: 0.2
  learnRate: 0.00908538854030219
  weightDecay: 1.5146136876672667e-05
  batchSize: 32
  epochs: 11
  optimizer: RMSprop
  use_scheduler: True




In [41]:
finalTrainDataset = TextDataset(trainEncode, yTrain)
finalValDataset = TextDataset(valEncode, yVal)

finalTrainLoader = DataLoader(finalTrainDataset, batch_size = best_batchSize, shuffle = True)
finalValLoader = DataLoader(finalValDataset, batch_size = best_batchSize, shuffle = False)

In [42]:
finalModel = feedforwardNeuralNetwork(
    vocabSize = vocabSize,
    embeddingDim = best_embeddingDim,
    hiddenUnit1 = best_hiddenUnit1,
    hiddenUnit2 = best_hiddenUnit2,
    dropoutRate = best_dropoutRate,
    numLabels = numLabels,
    padIndex = padIndex
).to(userDevice)

if best_optimizer == "Adam":
    finalOptimizer = optim.Adam(finalModel.parameters(), lr = best_learnRate, weight_decay = best_weightDecay)
elif best_optimizer == "RMSprop":
    finalOptimizer = optim.RMSprop(finalModel.parameters(), lr = best_learnRate, weight_decay = best_weightDecay)
else:
    finalOptimizer = optim.SGD(finalModel.parameters(), lr = best_learnRate, momentum = best_momentum, weight_decay = best_weightDecay)

finalCriterion = nn.BCEWithLogitsLoss(pos_weight = weight_tensor)

if best_use_scheduler:
    finalScheduler = optim.lr_scheduler.ReduceLROnPlateau(finalOptimizer, mode = "min", factor = 0.5, patience = 1)

In [43]:
bestValLossFinal = float("inf")
patienceCtr = 0
bestStateFinal = None

In [44]:
for epoch in range(1, best_epochs + 1):
    trainLoss = epochTrain(finalModel, finalTrainLoader, finalOptimizer, finalCriterion, userDevice)
    valLoss = evaluateModel(finalModel, finalValLoader,   finalCriterion, userDevice)
    print(f"Final Epoch {epoch:02d} | Train Loss: {trainLoss:.4f} | Val Loss: {valLoss:.4f}")

    if best_use_scheduler:
        finalScheduler.step(valLoss)

    if valLoss < bestValLossFinal:
        bestValLossFinal = valLoss
        patienceCtr = 0
        bestStateFinal = {k: v.cpu() for k, v in finalModel.state_dict().items()}
    else:
        patienceCtr += 1
        if patienceCtr >= 3:
            print(f"Early stopping at epoch {epoch}")
            break

Final Epoch 01 | Train Loss: 0.1890 | Val Loss: 0.1798
Final Epoch 02 | Train Loss: 0.1657 | Val Loss: 0.1726
Final Epoch 03 | Train Loss: 0.1530 | Val Loss: 0.1677
Final Epoch 04 | Train Loss: 0.1411 | Val Loss: 0.1740
Final Epoch 05 | Train Loss: 0.1295 | Val Loss: 0.1777
Final Epoch 06 | Train Loss: 0.1031 | Val Loss: 0.1888
Early stopping at epoch 6


In [45]:
finalModel.load_state_dict(bestStateFinal)

<All keys matched successfully>

In [46]:
finalModel.eval()
runningLoss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for batchInputs, batchLabels in finalValLoader:
        batchInputs = batchInputs.to(userDevice)
        batchLabels = batchLabels.to(userDevice)

        logits = finalModel(batchInputs)
        loss = finalCriterion(logits, batchLabels)
        runningLoss += loss.item() * batchInputs.size(0)

        preds = (torch.sigmoid(logits) >= 0.5).float()
        correct += (preds == batchLabels).sum().item()
        total += batchLabels.numel()

finalValLoss = runningLoss / len(finalValDataset)
finalValAcc  = correct / total
print(f"\nFinal Model -> Val Loss: {finalValLoss:.4f}, Val Accuracy: {finalValAcc:.4f}")


Final Model -> Val Loss: 0.1888, Val Accuracy: 0.7097


In [ ]:
os.makedirs("savedModel", exist_ok = True)
torch.save({
    "model_state_dict": finalModel.state_dict(),
    "wordtoindex": wordtoindex,
    "indextoword": indextoword,
    "padIndex": padIndex,
    "max_sequence_length": MAX_SEQUENCE_LENGTH
}, "savedModel/ffnn.pth")
print("Saved final model + vocab to savedModel/ffnn.pth")